In [1]:
%load_ext autoreload
%autoreload 1
%aimport transformations

transformations loaded, version: 0.1


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 999
pd.options.display.max_rows = 113

import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import StandardScaler, PowerTransformer

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

RS = 35577 # global random state seed
data_path = './data-raw/'

%aimport transformations
%aimport f

framework loaded, version: 0.1


In [3]:
(X, y) = f.from_pkl(f'data-processed/all_b1_b2.data.pkl')

X.shape
X.head(3)

y.shape
y.head(3)

(200, 36)

,x__amin,x__amax,x__sum,x__median,x__mean,x__std,x__var,x__q1,x__q3,x__iqr,x__kurtosis,x__skew,y__amin,y__amax,y__sum,y__median,y__mean,y__std,y__var,y__q1,y__q3,y__iqr,y__kurtosis,y__skew,z__amin,z__amax,z__sum,z__median,z__mean,z__std,z__var,z__q1,z__q3,z__iqr,z__kurtosis,z__skew
0,-6.716668,5.307868,-10988.070315,-0.079124,-0.134299,1.359987,1.849565,-0.944891,0.690446,1.635337,0.645580,-0.260035,-5.812267,5.638507,-2005.749440,-0.039563,-0.024515,1.090317,1.188792,-0.702004,0.622879,1.324883,0.845721,0.190106,-8.080924,8.244501,-15909.421691,-0.206307,-0.194449,1.503315,2.259956,-1.070594,0.754012,1.824606,0.873858,-0.226021
1,-12.103661,11.753020,-18844.323785,-0.175320,-0.239902,2.477716,6.139077,-1.522068,1.075231,2.597300,1.408079,-0.167536,-8.745936,12.452191,-318.869239,-0.134197,-0.004059,2.376120,5.645946,-1.459080,0.906782,2.365862,1.936212,0.887947,-10.193626,10.357203,-6543.554226,-0.110275,-0.083304,2.156482,4.650413,-1.262658,1.138140,2.400798,0.657898,0.065098
2,-7.678631,6.558420,-12546.141906,-0.079124,-0.136898,1.350739,1.824495,-0.752498,0.498054,1.250552,1.485456,-0.097227,-4.298115,5.259969,5780.428883,0.055072,0.063073,0.822131,0.675900,-0.323466,0.433610,0.757076,2.150199,0.258396,-10.289657,8.244501,12884.241287,0.177821,0.140587,1.480194,2.190974,-0.398371,0.850044,1.248415,3.049378,-0.710093


(200,)

0    1
1    1
2    1
Name: status, dtype: int64

# Models

In [4]:
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [25]:
all_models = [
    # lambda random_state: LogisticRegression(random_state=random_state),
    lambda random_state: KNeighborsClassifier(),
    lambda random_state: DecisionTreeClassifier(random_state=random_state),
    lambda random_state: RandomForestClassifier(random_state=random_state),
    lambda random_state: AdaBoostClassifier(random_state=random_state),
]

# CV

In [17]:
def cross_val(model_factory, X, y, cv_repeat_n = 1, model_repeat_n = 1, folds_n = 7):
    np.random.seed(RS)
    get_random = lambda  : np.random.randint(1, 10000)
    cv_scores = []
    
    for i_cv in range(cv_repeat_n):
        fold_scores = []
        cv_scores.append(fold_scores)
        
        for i_fold, (idx_train, idx_test) in enumerate(StratifiedKFold(n_splits=folds_n, shuffle=True, random_state=get_random()).split(X, y, y)):
            
            X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
            X_test, y_test = X.iloc[idx_test], y.iloc[idx_test]
            
            model_scores = []
            fold_scores.append(model_scores)

            for i_model in range(model_repeat_n):
                model = model_factory(get_random())
                model.fit(X_train, y_train)

                y_pred = model.predict_proba(X_test)[:, 1]
                score = roc_auc_score(y_test, y_pred)
                model_scores.append(score)
                
    model_name = model_factory(0).__class__.__name__
    return model_name, cv_repeat_n, folds_n, model_repeat_n, np.array(cv_scores)

In [40]:
def get_stats(results):
    idx = results[0]
    
    stats_ = results[4]
    return pd.DataFrame(
        {
            'folds_n' : results[2],    
            'cv_repeat_n' : results[1],            
            'model_repeat_n' : results[3],

            'min': stats_.min(),
            'mean': stats_.mean(),
            'max': stats_.max(),
            'std': stats_.std(),

            'fold_min': stats_.mean(axis=2).reshape(1,-1).min(),
            'fold_mean': stats_.mean(axis=2).reshape(1,-1).mean(),
            'fold_max': stats_.mean(axis=2).reshape(1,-1).max(),
            'fold_std': stats_.mean(axis=2).reshape(1,-1).std(),
        },
        index=[idx]
    )

## Single Run

In [20]:
%%time 

results = cross_val(
        lambda random_state: DecisionTreeClassifier(random_state=random_state),
        X, y, 
        cv_repeat_n = 1,
        model_repeat_n = 1,
    )

get_stats(results)

CPU times: user 55.8 ms, sys: 5.12 ms, total: 61 ms
Wall time: 58.6 ms


,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,1,7,1,0.642857,0.855102,0.966667,0.097111,0.642857,0.855102,0.966667,0.097111


In [21]:
%%time 

results = cross_val(
        lambda random_state: DecisionTreeClassifier(random_state=random_state),
        X, y, 
        cv_repeat_n = 1,
        model_repeat_n = 10,
    )

get_stats(results)

CPU times: user 355 ms, sys: 3.04 ms, total: 358 ms
Wall time: 354 ms


,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,1,7,10,0.642857,0.854898,0.966667,0.090684,0.653571,0.854898,0.922381,0.086868


In [22]:
%%time 

results = cross_val(
        lambda random_state: DecisionTreeClassifier(random_state=random_state),
        X, y, 
        cv_repeat_n = 10,
        model_repeat_n = 10,
    )

get_stats(results)

CPU times: user 3.34 s, sys: 15.7 ms, total: 3.36 s
Wall time: 3.36 s


,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,10,7,10,0.642857,0.859592,1.0,0.063023,0.653571,0.859592,1.0,0.056538


In [23]:
%%time 

results = cross_val(
        lambda random_state: RandomForestClassifier(random_state=random_state),
        X, y, 
        cv_repeat_n = 1,
        model_repeat_n = 1,
    )

get_stats(results)

CPU times: user 1.14 s, sys: 32 ms, total: 1.18 s
Wall time: 1.17 s


,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
RandomForestClassifier,1,7,1,0.946429,0.986176,1.0,0.018403,0.946429,0.986176,1.0,0.018403


In [15]:
%%time 

results = cross_val(
        lambda random_state: RandomForestClassifier(random_state=random_state),
        X, y, 
        cv_repeat_n = 1,
        model_repeat_n = 10,
    )

get_stats(results)

CPU times: user 10.1 s, sys: 256 ms, total: 10.4 s
Wall time: 10.4 s


,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
RandomForestClassifier,0.933673,0.983754,1.0,0.019589,0.94949,0.983754,1.0,0.01876


In [16]:
%%time 

results = cross_val(
        lambda random_state: RandomForestClassifier(random_state=random_state),
        X, y, 
        cv_repeat_n = 10,
        model_repeat_n = 10,
    )

get_stats(results)

CPU times: user 1min 40s, sys: 3.09 s, total: 1min 44s
Wall time: 1min 43s


,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
RandomForestClassifier,0.933333,0.984596,1.0,0.014544,0.94949,0.984596,1.0,0.013573


## Multiple Models

In [31]:
all_results = []

In [32]:
%%time 

for factory in all_models:
    results = cross_val(
            factory,
            X, y, 
            cv_repeat_n = 1,
            model_repeat_n = 1,
        )
    all_results.append(get_stats(results))

CPU times: user 4.43 s, sys: 173 ms, total: 4.6 s
Wall time: 1.84 s


In [33]:
%%time 

for factory in all_models:
    results = cross_val(
            factory,
            X, y, 
            cv_repeat_n = 1,
            model_repeat_n = 10,
        )
    all_results.append(get_stats(results))

CPU times: user 22.2 s, sys: 537 ms, total: 22.7 s
Wall time: 17.2 s


In [34]:
%%time 

for factory in all_models:
    results = cross_val(
            factory,
            X, y, 
            cv_repeat_n = 10,
            model_repeat_n = 10,
        )
    all_results.append(get_stats(results))

CPU times: user 3min 31s, sys: 5.09 s, total: 3min 36s
Wall time: 2min 52s


In [39]:
pd.concat(all_results).sort_index().round(2)

,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
AdaBoostClassifier,1,7,1,0.95,0.99,1.00,0.02,0.95,0.99,1.00,0.02
AdaBoostClassifier,1,7,10,0.95,0.99,1.00,0.02,0.95,0.99,1.00,0.02
AdaBoostClassifier,10,7,10,0.92,0.98,1.00,0.02,0.92,0.98,1.00,0.02
DecisionTreeClassifier,1,7,1,0.64,0.86,0.97,0.10,0.64,0.86,0.97,0.10
DecisionTreeClassifier,1,7,10,0.64,0.85,0.97,0.09,0.65,0.85,0.92,0.09
DecisionTreeClassifier,10,7,10,0.64,0.86,1.00,0.06,0.65,0.86,1.00,0.06
KNeighborsClassifier,1,7,1,0.88,0.95,0.99,0.04,0.88,0.95,0.99,0.04
KNeighborsClassifier,1,7,10,0.88,0.95,0.99,0.04,0.88,0.95,0.99,0.04
KNeighborsClassifier,10,7,10,0.85,0.94,1.00,0.04,0.85,0.94,1.00,0.04
RandomForestClassifier,1,7,1,0.95,0.99,1.00,0.02,0.95,0.99,1.00,0.02
